### Internal Knowledge Base Q&A Using Langchain & OpenAI
##### This example shows how to query an internal knowledge base stored in a GitHub repo as Markdown files.

##### This notebook is adapted from the Retrieval Question Answering with Sources example by Langchain.

In [ ]:
# !pip install langchain==0.0.123 # https://github.com/hwchase17/langchain/releases
# !pip install openai
# !pip install faiss-cpu

### Set up OPEN_API_KEY and necessary variables

In [22]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI API key here and hit enter:")

Paste your OpenAI API key here and hit enter:········


In [2]:
REPO_URL = "https://github.com/GovTechSG/developer.gov.sg"  # Source URL
DOCS_FOLDER = "docs"  # Folder to check out to
REPO_DOCUMENTS_PATH = "collections/_products/categories/devops/ship-hats"  # Set to "" to index the whole data folder
DOCUMENT_BASE_URL = "https://www.developer.tech.gov.sg/products/categories/devops/ship-hats"  # Actual URL
DATA_STORE_DIR = "data_store"

## Build the datastore 
*(Skip to next section to load data store from files if it has been saved locally to save cost of embeddings)*

### Clone the GitHub repo

In [8]:
!git clone $REPO_URL $DOCS_FOLDER

Cloning into 'docs'...
Updating files:   9% (189/1963)
Updating files:  10% (197/1963)
Updating files:  11% (216/1963)
Updating files:  12% (236/1963)
Updating files:  13% (256/1963)
Updating files:  14% (275/1963)
Updating files:  14% (279/1963)
Updating files:  15% (295/1963)
Updating files:  16% (315/1963)
Updating files:  17% (334/1963)
Updating files:  18% (354/1963)
Updating files:  18% (367/1963)
Updating files:  19% (373/1963)
Updating files:  20% (393/1963)
Updating files:  21% (413/1963)
Updating files:  21% (423/1963)
Updating files:  22% (432/1963)
Updating files:  23% (452/1963)
Updating files:  24% (472/1963)
Updating files:  25% (491/1963)
Updating files:  26% (511/1963)
Updating files:  27% (531/1963)
Updating files:  28% (550/1963)
Updating files:  28% (557/1963)
Updating files:  29% (570/1963)
Updating files:  30% (589/1963)
Updating files:  31% (609/1963)
Updating files:  32% (629/1963)
Updating files:  33% (648/1963)
Updating files:  34% (668/1963)
Updating files:  

### Load documents and split them into chunks for conversion to embeddings

In [9]:
import os
import pathlib
import re

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

name_filter = "**/*.md"
separator = "\n### "  # This separator assumes Markdown docs from the repo uses ### as logical main header most of the time
chunk_size_limit = 1000
max_chunk_overlap = 20

repo_path = pathlib.Path(os.path.join(DOCS_FOLDER, REPO_DOCUMENTS_PATH))
document_files = list(repo_path.glob(name_filter))

def convert_path_to_doc_url(doc_path):
  # Convert from relative path to actual document url
  return re.sub(f"{DOCS_FOLDER}/{REPO_DOCUMENTS_PATH}/(.*)\.[\w\d]+", f"{DOCUMENT_BASE_URL}/\\1", str(doc_path))

documents = [
    Document(
        page_content=open(file, "r").read(),
        metadata={"source": convert_path_to_doc_url(file)}
    )
    for file in document_files
]

text_splitter = CharacterTextSplitter(separator=separator, chunk_size=chunk_size_limit, chunk_overlap=max_chunk_overlap)
split_docs = text_splitter.split_documents(documents)

Created a chunk of size 1597, which is longer than the specified 1000
Created a chunk of size 1295, which is longer than the specified 1000
Created a chunk of size 2141, which is longer than the specified 1000


### (Optional) Check estimated tokens and costs

In [ ]:
#!pip install tiktoken

In [11]:
import tiktoken
# create a GPT-4 encoder instance
enc = tiktoken.encoding_for_model("gpt-4")

total_word_count = sum(len(doc.page_content.split()) for doc in split_docs)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in split_docs)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0004 / 1000}")


Total word count: 2065

Estimated tokens: 5230

Estimated cost of embedding: $0.002092


In [12]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(split_docs, embeddings)

### Verify content of Vector Store with a sample query

In [13]:
from IPython.display import display, Markdown

search_result = vector_store.similarity_search_with_score("What is SHIP-HATS?")
search_result

line_separator = "\n"# {line_separator}Source: {r[0].metadata['source']}{line_separator}Score:{r[1]}{line_separator}
display(Markdown(f"""
## Search results:{line_separator}
{line_separator.join([
  f'''
  ### Source:{line_separator}{r[0].metadata['source']}{line_separator}
  #### Score:{line_separator}{r[1]}{line_separator}
  #### Content:{line_separator}{r[0].page_content}{line_separator}
  '''
  for r in search_result
])}
"""))


## Search results:


  ### Source:
docs\collections\_products\categories\devops\ship-hats\overview.md

  #### Score:
0.16688910126686096

  #### Content:
What is SHIP-HATS?

**SHIP (Secure Hybrid Integration Pipeline)-HATS (Hive Agile Testing Solutions)** is Continuous Integration/Continuous Delivery [CI/CD](https://en.wikipedia.org/wiki/CI/CD){:target="_blank"} component within SG Government Tech Stack (SGTS) with security and governance guardrails that enables developers to plan, build, test and deploy code to production.

This is a multi-tenanted Software-as-a-Service (SaaS) based end-to-end CI/CD for all applications that is classified as RESTRICTED and below.

[Why is CI/CD important?](https://youtu.be/RlZCyexsJBc?t=260){:Target="_blank"}

  

  ### Source:
docs\collections\_products\categories\devops\ship-hats\overview.md

  #### Score:
0.23639556765556335

  #### Content:
Benefits of SHIP-HATS

1. **Policy Compliance:** Standardised development tools and environment that is set up in compliance with **Application Infrastructure Architecture Standard (AIAS)** and **Instruction Manual 8 on Information Technology (IM8)** policies.
2. **Better Quality:** Automated security testing detects vulnerabilities early in the development cycle helping to deliver high quality applications. SHIP-HATS allows agencies to follow prescriptive workflow templates and share knowledge among themselves.
3. **Visibility & Transparency:** Agencies will always have access to source code and hence a better understanding of how the project is progressing.
4. **Shortened Time-to-market:** Agencies can shorten the time to market by leveraging on SHIP-HATS CI/CD tools with predefined & re-usable configurations into their system and do not have to invest in resources/time to do procurement with different commercial providers. Agencies also do not have to invest resources to re-train staff who were re-deployed across agencies.
5. **Economies of Scale:** SHIP-HATS purchases licenses in bulk and redistributes them for use in smaller quantities to agencies and is offered as a complete package with no hidden cost.
6. **GovTech Managed:** The CI/CD tools are procured and managed by GovTech letting you focus on your core application.
7. **Performance Management Dashboard:** Value-Stream Measurement capabilities that allow Agencies to capture key industry metrics, such as lead time to deployment and deployment frequency, to monitor the effectiveness of their DevSecOps practices

  

  ### Source:
docs\collections\_products\categories\devops\ship-hats\resources.md

  #### Score:
0.23909235000610352

  #### Content:
SHIP-HATS 

| Developers |  Project Managers, Business Analysts  |
| :------------- | ----------------------------------------------------------------------------------------- |
| [SHIP-HATS Architectural Approach](https://www.youtube.com/watch?v=yiD4--KSdTI){:target="_blank"}<br />[Roadmap](./overview#roadmap)<br /><br />[User Roles & Permissions](https://docs.developer.gov.sg/docs/ship-hats-documentation/#/user-roles-permissions){:target="_blank"}<br /> [Security Testing 101](https://www.youtube.com/watch?v=SVomPCqKGM4){:target="_blank"}<br />[Continuous Delivery](https://www.youtube.com/watch?v=DMMhqLKHLx0){:target="_blank"} | [SHIP-HATS Overview for Newbies](https://www.youtube.com/watch?v=SVomPCqKGM4){:target="_blank"}<br />[Understanding Subscription](./subscriptions){:target="_blank"}<br /><br />[Request trial (for Public Officers)](./subscription#11-can-i-request-for-a-trial-subscription){:target="_blank"}

  

  ### Source:
docs\collections\_products\categories\devops\ship-hats\training\tools.md

  #### Score:
0.2773140072822571

  #### Content:
---
title: Tools in SHIP-HATS
layout: layout-page-sidenav
description: insert description.
published: false
---

### Overview
 
**Commercially Off the Shelf (COTS)** tools are available on SHIP-HATS with right security and compliance settings. Here are curated links to documentation and tutorials to first learn the tools offered under **SHIP-HATS**. Note this is not specific to SHIP-HATS but a pre-cursor so you can use these tools within SHIP-HATS effectively.

  


## (Optional) Save vector store to files and download/save in another location for reuse

In [14]:
vector_store.save_local(DATA_STORE_DIR)
# Download the files `$DATA_STORE_DIR/index.faiss` and `$DATA_STORE_DIR/index.pkl` to local

#### To load the Vector Store from files:

In [15]:
# Upload the files `$DATA_STORE_DIR/index.faiss` and `$DATA_STORE_DIR/index.pkl` to local
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

if os.path.exists(DATA_STORE_DIR):
  vector_store = FAISS.load_local(
      DATA_STORE_DIR,
      OpenAIEmbeddings()
  )
else:
  print(f"Missing files. Upload index.faiss and index.pkl files to {DATA_STORE_DIR} directory first")

## Query using the vector store with ChatGPT integration
### Set up the chat model and specific prompt

In [16]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question.
Take note of the sources and include them in the answer in the format: "SOURCES: source1 source2", use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=256)  # Modify model_name if you have access to GPT-4
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    verbose = True,
    chain_type_kwargs=chain_type_kwargs
)

from IPython.display import display, Markdown
def print_result(result):
  output_text = f"""### Question: 
  {query}
  ### Answer: 
  {result['answer']}
  ### Sources: 
  {result['sources']}
  ### All relevant sources:
  {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
  """
  display(Markdown(output_text))

#### Use the chain to query

In [18]:
query = "What is SHIP-HATS?"
result = chain(query)
print_result(result)

### Question: 
  What is SHIP-HATS?
  ### Answer: 
  SHIP (Secure Hybrid Integration Pipeline)-HATS (Hive Agile Testing Solutions) is a Continuous Integration/Continuous Delivery (CI/CD) component within SG Government Tech Stack (SGTS) that enables developers to plan, build, test, and deploy code to production. It is a multi-tenanted Software-as-a-Service (SaaS) based end-to-end CI/CD for all applications that is classified as RESTRICTED and below. It provides security and governance guardrails to ensure policy compliance, better quality, visibility & transparency, shortened time-to-market, economies of scale, GovTech managed, and performance management dashboard. 
  ### Sources: 
  docs\collections\_products\categories\devops\ship-hats\overview.md
  ### All relevant sources:
  docs\collections\_products\categories\devops\ship-hats\resources.md docs\collections\_products\categories\devops\ship-hats\training\tools.md docs\collections\_products\categories\devops\ship-hats\overview.md
  

In [21]:
import langchain
langchain.debug=True

query = "What is SHIP-HATS?"
result = chain(query)
print_result(result)
langchain.debug=False

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is SHIP-HATS?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is SHIP-HATS?",
  "summaries": "Content: What is SHIP-HATS?\n\n**SHIP (Secure Hybrid Integration Pipeline)-HATS (Hive Agile Testing Solutions)** is Continuous Integration/Continuous Delivery [CI/CD](https://en.wikipedia.org/wiki/CI/CD){:target=\"_blank\"} component within SG Government Tech Stack (SGTS) with security and governance guardrails that enables developers to plan, build, test and deploy code to production.\n\nThis is a multi-tenanted Software-as-a-Service (SaaS) based end-to-end CI/CD for all applications that is classified as RESTRICTED and below.\n\n[Why is CI/CD important?](https:/

[llm/end] [1:chain:RetrievalQAWithSourcesChain > 2:chain:StuffDocumentsChain > 3:chain:LLMChain > 4:llm:ChatOpenAI] [9.21s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "SHIP (Secure Hybrid Integration Pipeline)-HATS (Hive Agile Testing Solutions) is a Continuous Integration/Continuous Delivery (CI/CD) component within SG Government Tech Stack (SGTS) that enables developers to plan, build, test, and deploy code to production. It is a multi-tenanted Software-as-a-Service (SaaS) based end-to-end CI/CD for all applications that is classified as RESTRICTED and below. It provides security and governance guardrails to ensure policy compliance, better quality, visibility & transparency, shortened time-to-market, economies of scale, GovTech managed, and performance management dashboard. SOURCES: docs\\collections\\_products\\categories\\devops\\ship-hats\\overview.md",
        "generation_info": null,
        "message": {
          "content": "SHIP (Secure Hy

### Question: 
  What is SHIP-HATS?
  ### Answer: 
  SHIP (Secure Hybrid Integration Pipeline)-HATS (Hive Agile Testing Solutions) is a Continuous Integration/Continuous Delivery (CI/CD) component within SG Government Tech Stack (SGTS) that enables developers to plan, build, test, and deploy code to production. It is a multi-tenanted Software-as-a-Service (SaaS) based end-to-end CI/CD for all applications that is classified as RESTRICTED and below. It provides security and governance guardrails to ensure policy compliance, better quality, visibility & transparency, shortened time-to-market, economies of scale, GovTech managed, and performance management dashboard. 
  ### Sources: 
  docs\collections\_products\categories\devops\ship-hats\overview.md
  ### All relevant sources:
  docs\collections\_products\categories\devops\ship-hats\resources.md docs\collections\_products\categories\devops\ship-hats\training\tools.md docs\collections\_products\categories\devops\ship-hats\overview.md
  